In [13]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime, timedelta
import time

In [14]:
# Load environment variables from .env file
load_dotenv()

# CoinMarketCap API URL
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

# Fetch API key from environment variables
api_key = os.getenv('COINMARKETCAP_API_KEY')

In [15]:
# Function to fetch and process data
def fetch_process_data():
        # Parameters for the request
        parameters = {
            'start': '1',
            'limit': '100',  # Fetching top 100 cryptocurrencies
            'convert': 'USD'
        }

        # Headers containing your API key
        headers = {
            'Accepts': 'application/json',
            'X-CMC_PRO_API_KEY': api_key,
        }

        # Make a GET request to the CoinMarketCap API
        response = requests.get(url, headers=headers, params=parameters)
        
        # Check for HTTP errors
        response.raise_for_status()
        
        # Load data into pandas DataFrame
        data = response.json()
        df = pd.json_normalize(data['data'])  # Normalize JSON data into DataFrame

        # Filter out stablecoins (assuming stablecoins have 'True' in 'platform.name' field)
        df_filtered = df[~df['tags'].apply(lambda x: 'Stablecoin' in x if isinstance(x, list) else False)]

        # Sort by market capitalization and select the top 10 cryptocurrencies
        df_filtered_sorted = df_filtered.sort_values(by='quote.USD.market_cap', ascending=False).head(10)

        # Calculate weights based on market capitalization
        total_market_cap = df_filtered_sorted['quote.USD.market_cap'].sum()
        df_filtered_sorted['weight'] = df_filtered_sorted['quote.USD.market_cap'] / total_market_cap

        # Create an index DataFrame
        index_df = df_filtered_sorted[['name', 'symbol', 'weight']]

        return index_df, total_market_cap

# Function to update the index fund monthly
def update_index_fund():
    # Initial fetch and process data
    index_df, prev_total_market_cap = fetch_process_data()

    if index_df is None:
        return

    # Print initial index
    print("Top 10 Cryptocurrencies by Market Capitalization:")
    print(index_df)

    # Loop to simulate monthly updates
    while True:
        # Wait for the next month
        next_month = datetime.now() + timedelta(days=30)
        print(f"\nWaiting until {next_month.strftime('%Y-%m-%d %H:%M:%S')} for next update...\n")
        time.sleep((next_month - datetime.now()).total_seconds())

        # Fetch and process updated data
        index_df, total_market_cap = fetch_process_data()

        if index_df is None:
            continue

        # Calculate updated weights based on the new total market capitalization
        index_df['weight'] = index_df['quote.USD.market_cap'] / total_market_cap

        # Print updated index
        print(f"Top 10 Cryptocurrencies by Market Capitalization as of {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}:")
        print(index_df)

        # Update previous total market capitalization for the next iteration
        prev_total_market_cap = total_market_cap

# Start the monthly update process
update_index_fund()


Top 10 Cryptocurrencies by Market Capitalization:
          name symbol    weight
0      Bitcoin    BTC  0.607994
1     Ethereum    ETH  0.204847
2  Tether USDt   USDT  0.057246
3          BNB    BNB  0.042742
4       Solana    SOL  0.031767
5         USDC   USDC  0.016555
6          XRP    XRP  0.013373
7      Toncoin    TON  0.009463
8     Dogecoin   DOGE  0.008975
9      Cardano    ADA  0.007037

Waiting until 2024-07-25 10:31:42 for next update...



KeyboardInterrupt: 